In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
"""!mkdir /content/drive/MyDrive/hindi_english
!unzip /content/drive/MyDrive/Hindi_English_Truncated_Corpus.csv.zip -d /content/drive/MyDrive/hindi_english"""

'!mkdir /content/drive/MyDrive/hindi_english\n!unzip /content/drive/MyDrive/Hindi_English_Truncated_Corpus.csv.zip -d /content/drive/MyDrive/hindi_english'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import re
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)
lines=pd.read_csv("/content/drive/MyDrive/hindi_english/Hindi_English_Truncated_Corpus.csv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [ ]:
lines

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
37863,ted,it seems that with or without us,START_ हमारे साथ या हमारे बिना भी _END,7,8
83895,tides,getting help,START_ सहायता प्राप्त करना _END,2,5
46957,ted,so that their children too can live those lives of great purpose,START_ जिससे कि उनके बच्चे भी किसी उद्देश्य या प्रयोजन से भरा जीवन जी सकें। _END,12,16
60041,ted,so im excited about how i can bring the technology to the masses,START_ तो मैं बहुत उत्सुक हूँ कि कैसे मैं इस तकनीक को आम लोगों तक पहुँचा सकूँ _END,13,18
120175,tides,the sudra is like a servant to the brahman taking care of his affairs and serving him,START_ शूद्र ब्राह्मण के सेवकजैसा होता है जो उसके कामकाज की देखभाल और उसकी सेवा करता है _END,17,18
...,...,...,...,...,...
71869,tides,i have always been acting independently without having cared for your approval or disapproval ”,START_ मैं आपकी सहमति या असहमति के बिना यहां स्वतंत्रतापूर्वक काम करता रहा हूं ” _END,15,16
63342,ted,but what makes voodoo so interesting,START_ लेकिन क्या चीज जादूटोने को इतना रूचिकारक बनाती है _END,6,11
29935,indic2012,listen here,START_ यहां सुनियेः _END,2,4
111449,indic2012,long time prevalent belief of its purification has scientific proof,START_ लंबे समय से प्रचलित इसकी शुद्धीकरण की मान्यता का वैज्ञानिक आधार भी है। _END,10,15


# removing null english and hindi lines

In [ ]:
lines=lines[~pd.isnull(lines['english_sentence'])]
lines=lines[~pd.isnull(lines['hindi_sentence'])]

# selectimng random 100000 samples

In [ ]:
lines=lines.sample(n=100000,random_state=42)
lines.shape

(100000, 3)

In [ ]:
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.lower())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

In [ ]:
import string
ex = set(string.punctuation)
lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in ex))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in ex))

In [ ]:
digits='1234567890'
remove_digits = str.maketrans('', '', digits)
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))


In [ ]:
lines.shape

(100000, 3)

In [ ]:
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.strip())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))

In [ ]:
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [ ]:
lines

,source,english_sentence,hindi_sentence
3556,tides,he declares the result and reports it to the election commission and to the secretarygeneral of the concerned house,START_ वही परिणाम की घोषणा करता है और निर्वाचन आयोग को और संबद्ध सदन के महासचिव को उसकी सूचना देता है _END
25899,ted,was a little uncomfortable for them,START_ थोडा कठिन था। _END
90924,tides,a multipurpose auditorium a branch of the state bank of india a large banquet hall private dining rooms and refreshment rooms including a milk bar are also located on the ground floor,START_ बहुउद्देशीय सभागार भारतीय स्टेट बैंक की शाखा एक विशाल बैंकट हाल प्राइवेट डाइनिंग रूम और जलपान गृह भी जिनमें एक मिल्क बार सम्मिलित है भूमि तल पर उपस्थित हैं _END
78213,tides,no fees is to be paid for filing the appeal to this tribunal,START_ अधिकरण में अपील करने के लिए कोई फीस नहीं देनी पड़ती _END
96955,indic2012,headind kaun banega crorepati,START_ शीर्षक कौन बनेगा करोड़पति kaun banega crorepati _END
...,...,...,...
75226,indic2012,he left kashi for magahar and he died there only in,START_ काशी छोड़ मगहर चले गये और सन् के आस पास वहीं देह त्याग किया। _END
18651,indic2012,pakistan muslim leag,START_ पाकिस्तान मुस्लिम लीग _END
35035,tides,minerals minerals such as magnesium copper and zinc are receiving increasing attention from researchers for their effect on heart disease obesity is defined as being about per cent of your ideal body weight as judged from tables provided lrb see appendix rrb,START_ खनिज हृदय रोग पर अपने प्रभाव के लिए मैग़्नीशियम तांबा और जिंक जैसे खनिज अनुसंधानकर्ताओं का लगातार ज़्यादा ध्यान आकर्षित कर रहे हैंशारीरिक भार पर नियंत्रण दी गयी सारणियों परिशिष्ट के आधार पर आपके शरीर के आदर्श भार से प्रतिशत अधिक भार को मोटापा _END
105680,ted,and over time we have grown into a network of five hospitals,START_ और धीरेधीरे हम पाँच अस्पतालों के समूह में विकसित हो गये _END


In [ ]:
all_eng_words=set()
for eng in lines['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [ ]:
len(all_eng_words)

64017

In [ ]:
len(all_hindi_words)

72923

In [ ]:
lines['length_eng_sentence']=lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [ ]:
lines[lines['length_eng_sentence']>30].shape

(9547, 5)

In [ ]:
lines

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
3556,tides,he declares the result and reports it to the election commission and to the secretarygeneral of the concerned house,START_ वही परिणाम की घोषणा करता है और निर्वाचन आयोग को और संबद्ध सदन के महासचिव को उसकी सूचना देता है _END,19,22
25899,ted,was a little uncomfortable for them,START_ थोडा कठिन था। _END,6,5
90924,tides,a multipurpose auditorium a branch of the state bank of india a large banquet hall private dining rooms and refreshment rooms including a milk bar are also located on the ground floor,START_ बहुउद्देशीय सभागार भारतीय स्टेट बैंक की शाखा एक विशाल बैंकट हाल प्राइवेट डाइनिंग रूम और जलपान गृह भी जिनमें एक मिल्क बार सम्मिलित है भूमि तल पर उपस्थित हैं _END,32,31
78213,tides,no fees is to be paid for filing the appeal to this tribunal,START_ अधिकरण में अपील करने के लिए कोई फीस नहीं देनी पड़ती _END,13,13
96955,indic2012,headind kaun banega crorepati,START_ शीर्षक कौन बनेगा करोड़पति kaun banega crorepati _END,4,9
...,...,...,...,...,...
75226,indic2012,he left kashi for magahar and he died there only in,START_ काशी छोड़ मगहर चले गये और सन् के आस पास वहीं देह त्याग किया। _END,11,16
18651,indic2012,pakistan muslim leag,START_ पाकिस्तान मुस्लिम लीग _END,3,5
35035,tides,minerals minerals such as magnesium copper and zinc are receiving increasing attention from researchers for their effect on heart disease obesity is defined as being about per cent of your ideal body weight as judged from tables provided lrb see appendix rrb,START_ खनिज हृदय रोग पर अपने प्रभाव के लिए मैग़्नीशियम तांबा और जिंक जैसे खनिज अनुसंधानकर्ताओं का लगातार ज़्यादा ध्यान आकर्षित कर रहे हैंशारीरिक भार पर नियंत्रण दी गयी सारणियों परिशिष्ट के आधार पर आपके शरीर के आदर्श भार से प्रतिशत अधिक भार को मोटापा _END,42,46
105680,ted,and over time we have grown into a network of five hospitals,START_ और धीरेधीरे हम पाँच अस्पतालों के समूह में विकसित हो गये _END,12,13


# The data is still a dataframe

In [ ]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]
lines.shape

(66853, 5)

In [ ]:
print("maximum length of Hindi Sentence ",max(lines['length_hin_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [ ]:
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

In [ ]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(64017, 72923)

In [ ]:
num_decoder_tokens += 1

In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [ ]:
lines = shuffle(lines)
lines

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
37863,ted,it seems that with or without us,START_ हमारे साथ या हमारे बिना भी _END,7,8
83895,tides,getting help,START_ सहायता प्राप्त करना _END,2,5
46957,ted,so that their children too can live those lives of great purpose,START_ जिससे कि उनके बच्चे भी किसी उद्देश्य या प्रयोजन से भरा जीवन जी सकें। _END,12,16
60041,ted,so im excited about how i can bring the technology to the masses,START_ तो मैं बहुत उत्सुक हूँ कि कैसे मैं इस तकनीक को आम लोगों तक पहुँचा सकूँ _END,13,18
120175,tides,the sudra is like a servant to the brahman taking care of his affairs and serving him,START_ शूद्र ब्राह्मण के सेवकजैसा होता है जो उसके कामकाज की देखभाल और उसकी सेवा करता है _END,17,18
...,...,...,...,...,...
71869,tides,i have always been acting independently without having cared for your approval or disapproval ”,START_ मैं आपकी सहमति या असहमति के बिना यहां स्वतंत्रतापूर्वक काम करता रहा हूं ” _END,15,16
63342,ted,but what makes voodoo so interesting,START_ लेकिन क्या चीज जादूटोने को इतना रूचिकारक बनाती है _END,6,11
29935,indic2012,listen here,START_ यहां सुनियेः _END,2,4
111449,indic2012,long time prevalent belief of its purification has scientific proof,START_ लंबे समय से प्रचलित इसकी शुद्धीकरण की मान्यता का वैज्ञानिक आधार भी है। _END,10,15


In [ ]:
X, y = lines['english_sentence'], lines['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.02,random_state=42)
X_train.shape, X_test.shape

((65515,), (1338,))

In [ ]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [ ]:
!ls

drive  sample_data  X_test.pkl	X_train.pkl


In [ ]:
"""!mv /content/X_test.pkl /content/drive/MyDrive/hindi_english
!mv /content/X_train.pkl /content/drive/MyDrive/hindi_english"""

'!mv /content/X_test.pkl /content/drive/MyDrive/hindi_english\n!mv /content/X_train.pkl /content/drive/MyDrive/hindi_english'

In [ ]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word]
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] 
                    if t>0:
                        
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [ ]:
latent_dim=300

In [ ]:
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    19205100    ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    21877200    ['input_2[0][0]']                
                                                                                              

In [ ]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 50

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6143160741040657336
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11320098816
locality {
  bus_id: 1
  links {
  }
}
incarnation: 746305218794097920
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
xla_global_id: 416903419
]


In [ ]:
!cd  /content/drive/MyDrive/hindi_english

In [ ]:
"""from keras.callbacks import EarlyStopping, ModelCheckpoint
!cd /content/drive/MyDrive/hindi_english
early_stop = EarlyStopping(monitor='loss', min_delta=0.001, patience=3, mode='min', verbose=1)
checkpoint = ModelCheckpoint('model_best_weights.h5', monitor='loss', verbose=1, save_best_only=True, mode='min', period=1)
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs, callbacks = [early_stop,checkpoint],
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size,)"""

"from keras.callbacks import EarlyStopping, ModelCheckpoint\n!cd /content/drive/MyDrive/hindi_english\nearly_stop = EarlyStopping(monitor='loss', min_delta=0.001, patience=3, mode='min', verbose=1)\ncheckpoint = ModelCheckpoint('model_best_weights.h5', monitor='loss', verbose=1, save_best_only=True, mode='min', period=1)\nmodel.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),\n                    steps_per_epoch = train_samples//batch_size,\n                    epochs=epochs, callbacks = [early_stop,checkpoint],\n                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),\n                    validation_steps = val_samples//batch_size,)"

In [ ]:
"""model.save_weights('/content/drive/MyDrive/hindi_english/model_best_weights.h5')
!mv /content/nmt_weights.h5 /content/drive/MyDrive/hindi_english
!mv /content/model_best_weights.h5 /content/drive/MyDrive/hindi_english"""

"model.save_weights('/content/drive/MyDrive/hindi_english/model_best_weights.h5')\n!mv /content/nmt_weights.h5 /content/drive/MyDrive/hindi_english\n!mv /content/model_best_weights.h5 /content/drive/MyDrive/hindi_english"

In [ ]:
model.load_weights("/content/drive/MyDrive/hindi_english/model_best_weights.h5")

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence



In [ ]:


train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1



In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: because you see the greeks believed
Actual Hindi Translation:  देखिय ग्रीक ये मानते थे 
Predicted Hindi Translation:  देखिय ग्रीक ये मानते थे 
